# XGB Classifier with Wrapper-Based FS

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [1]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_unbalanced_with_lexicalcontent.csv")   

dataset.head()



,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
dataset.head()

,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,js_eval_count,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,14,36,0,0.000000,1186,5.269303,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,415,3952,11450,4.683930,26155,4.565537,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5715,88450,420,5.833333,339327,4.656704,13,10,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1461,6292,3049,4.515907,89249,5.149580,11,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,43,273,0,0.000000,1530,4.754726,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

#### 2. Preprocessing (Balancing)

In [4]:
dataset['url_type'].value_counts()

url_type
0    120080
1     62334
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [5]:
important_features_wrapper_12 = ['blank_lines_count', 
                                 'word_count', 
                                 'js_count', 
                                 'js_link_count', 
                                 'js_winopen_count', 
                                 'meta_tag_count', 
                                 'anchor_tag_count', 
                                 'input_tag_count', 
                                 'has_free_in_html', 
                                 'url_length', 
                                 'url_domain_entropy', 
                                 'url_number_of_digits', 
                                 'url_path_length', 
                                 'url_host_length', 
                                 'url_number_of_subdirectories', 
                                 'url_has_port', 
                                 'url_number_of_fragments', 
                                 'url_num_periods', 
                                 'has_ebayisapi_in_string', 
                                 'has_logon_in_string', 
                                 'has_signon_in_string', 
                                 'has_loginasp_in_string', 
                                 'has_exe_in_string', 
                                 'has_zip_in_string', 
                                 'has_paypal_in_string']


important_features_wrapper_33 = ['blank_lines_count', 
                                 'word_count', 
                                 'js_count', 
                                 'js_find_count', 
                                 'js_link_count', 
                                 'js_winopen_count', 
                                 'title_tag_presence', 
                                 'meta_tag_count', 
                                 'anchor_tag_count', 
                                 'applet_tag_count', 
                                 'input_tag_count', 
                                 'has_free_in_html', 
                                 'has_access_in_html', 
                                 'url_length', 
                                 'url_ip_in_domain', 
                                 'url_domain_entropy', 
                                 'url_is_digits_in_domain', 
                                 'url_number_of_digits', 
                                 'url_string_entropy', 
                                 'url_path_length', 
                                 'url_host_length', 
                                 'url_number_of_subdirectories', 
                                 'url_num_subdomain', 
                                 'url_has_port', 
                                 'url_number_of_fragments', 
                                 'url_num_periods', 
                                 'url_num_equal', 
                                 'url_num_open_parenthesis', 
                                 'url_num_close_parenthesis', 
                                 'url_num_ampersand', 
                                 'url_num_at', 
                                 'has_account_in_string', 
                                 'has_webscr_in_string', 
                                 'has_ebayisapi_in_string', 
                                 'has_signin_in_string', 
                                 'has_banking_in_string', 
                                 'has_confirm_in_string', 
                                 'has_logon_in_string', 
                                 'has_signon_in_string', 
                                 'has_loginasp_in_string', 
                                 'has_loginphp_in_string', 
                                 'has_exe_in_string', 
                                 'has_zip_in_string', 
                                 'has_rar_in_string', 
                                 'has_jpg_in_string', 
                                 'has_gif_in_string', 
                                 'has_viewerphp_in_string', 
                                 'has_getImageasp_in_string', 
                                 'has_plugins_in_string', 
                                 'has_paypal_in_string', 
                                 'has_dbsysphp_in_string', 
                                 'has_configbin_in_string', 
                                 'has_downloadphp_in_string', 
                                 'has_payment_in_string', 
                                 'has_files_in_string', 
                                 'has_shopping_in_string', 
                                 'has_mailphp_in_string', 
                                 'has_jar_in_string', 
                                 'has_swf_in_string', 
                                 'has_cgi_in_string', 
                                 'has_php_in_string', 
                                 'has_abuse_in_string', 
                                 'has_bin_in_string', 
                                 'has_update_in_string', 
                                 'has_verification_in_string']


X_test_12 = x_test[important_features_wrapper_12]
X_train_12 = x_train[important_features_wrapper_12]

X_test_33 = x_test[important_features_wrapper_33]
X_train_33 = x_train[important_features_wrapper_33]

X_test_33.head()

,blank_lines_count,word_count,js_count,js_find_count,js_link_count,js_winopen_count,title_tag_presence,meta_tag_count,anchor_tag_count,applet_tag_count,...,has_shopping_in_string,has_mailphp_in_string,has_jar_in_string,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_bin_in_string,has_update_in_string,has_verification_in_string
170772,377,165,11,0,1,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
49467,89,8,49,1,1,0,1,6,0,0,...,0,0,0,0,0,0,0,0,0,0
18363,3538,16323,87,0,2,0,1,43,0,0,...,0,0,0,0,0,0,0,0,0,0
62898,2807,6330,12,0,1,0,1,22,0,0,...,0,0,0,0,0,0,0,0,0,0
8428,5557,25987,10,3,4,0,1,13,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Hyper-parameter Tuning

##### 12 Features (Purely Lexical)

In [6]:
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

# Define the objective function for Optuna
def objective_12(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_12, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_12 = optuna.create_study(direction='minimize')
study_12.optimize(objective_12, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_12 = study_12.best_params
best_error = study_12.best_value
print("Best Hyperparameters (12 Features): ", best_params_12)
print("Best Error (12 Features): ", best_error)

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-04-02 15:06:50,549] A new study created in memory with name: no-name-2e6bd9e7-a228-4c8f-92f1-62683e23d351


[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.65995
[3]	validation-error:0.65995
[4]	validation-error:0.65995
[5]	validation-error:0.65995
[6]	validation-error:0.65995
[7]	validation-error:0.65995
[8]	validation-error:0.65995
[9]	validation-error:0.65995
[10]	validation-error:0.65995
[11]	validation-error:0.65995
[12]	validation-error:0.65995
[13]	validation-error:0.65995
[14]	validation-error:0.65995
[15]	validation-error:0.65995
[16]	validation-error:0.65235
[17]	validation-error:0.31113


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:06:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[18]	validation-error:0.30024
[19]	validation-error:0.28931
[20]	validation-error:0.26491
[21]	validation-error:0.26138
[22]	validation-error:0.19365
[23]	validation-error:0.14232
[24]	validation-error:0.12982
[25]	validation-error:0.12694
[26]	validation-error:0.12461
[27]	validation-error:0.12225
[28]	validation-error:0.12091
[29]	validation-error:0.12077
[30]	validation-error:0.11731
[31]	validation-error:0.11731
[32]	validation-error:0.11642
[33]	validation-error:0.11635
[34]	validation-error:0.11615
[35]	validation-error:0.11389
[36]	validation-error:0.11392
[37]	validation-error:0.11125
[38]	validation-error:0.11008
[39]	validation-error:0.10840
[40]	validation-error:0.10772
[41]	validation-error:0.10491
[42]	validation-error:0.10327
[43]	validation-error:0.10275
[44]	validation-error:0.09977
[45]	validation-error:0.09823
[46]	validation-error:0.09734
[47]	validation-error:0.09741
[48]	validation-error:0.09641
[49]	validation-error:0.09607
[50]	validation-error:0.09491
[51]	valid

[I 2024-04-02 15:07:05,702] Trial 0 finished with value: 0.20752618852281776 and parameters: {'eta': 0.034477415543551576, 'max_depth': 4, 'subsample': 0.7333584939740789, 'colsample_bytree': 0.8967190074261338, 'gamma': 3.161549110834766, 'min_child_weight': 0.45516791754979535, 'lambda': 8.771644999837628, 'alpha': 3.8001666431140144}. Best is trial 0 with value: 0.20752618852281776.


...

[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.20149


[I 2024-04-02 15:09:28,875] Trial 98 pruned. Trial was pruned at iteration 2.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:09:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.16504


[I 2024-04-02 15:09:29,102] Trial 99 pruned. Trial was pruned at iteration 2.


Best Hyperparameters (12 Features):  {'eta': 0.21019153698092585, 'max_depth': 10, 'subsample': 0.9321646856091291, 'colsample_bytree': 0.8325501350820024, 'gamma': 0.7374044077961104, 'min_child_weight': 5.5166658745205455, 'lambda': 3.895007719834459, 'alpha': 4.359152785531455}
Best Error (12 Features):  0.18903726494787432


##### 33 Features (Purely Lexical)

In [7]:
# Define the objective function for Optuna
def objective_33(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_33, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_33 = optuna.create_study(direction='minimize')
study_33.optimize(objective_33, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_33 = study_33.best_params
best_error_33 = study_33.best_value
print("Best Hyperparameters (33 Features): ", best_params_33)
print("Best Error (33 Features): ", best_error)

[I 2024-04-02 15:09:29,127] A new study created in memory with name: no-name-bf5d7715-c6d0-45f2-9d49-e5ce0badbd93


[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.65995
[3]	validation-error:0.65995
[4]	validation-error:0.65995
[5]	validation-error:0.65995


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:09:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[6]	validation-error:0.65995
[7]	validation-error:0.65995
[8]	validation-error:0.65995
[9]	validation-error:0.39826
[10]	validation-error:0.18162
[11]	validation-error:0.12745
[12]	validation-error:0.10464
[13]	validation-error:0.08624
[14]	validation-error:0.07867
[15]	validation-error:0.07298
[16]	validation-error:0.06866
[17]	validation-error:0.06623
[18]	validation-error:0.06424
[19]	validation-error:0.06188
[20]	validation-error:0.06010
[21]	validation-error:0.05903
[22]	validation-error:0.05807
[23]	validation-error:0.05708
[24]	validation-error:0.05581
[25]	validation-error:0.05523
[26]	validation-error:0.05461
[27]	validation-error:0.05369
[28]	validation-error:0.05269
[29]	validation-error:0.05225
[30]	validation-error:0.05160
[31]	validation-error:0.05126
[32]	validation-error:0.05050
[33]	validation-error:0.05013
[34]	validation-error:0.04968
[35]	validation-error:0.04917
[36]	validation-error:0.04910
[37]	validation-error:0.04879
[38]	validation-error:0.04886
[39]	validatio

[I 2024-04-02 15:09:44,415] Trial 0 finished with value: 0.18295848390224673 and parameters: {'eta': 0.053759444139250497, 'max_depth': 10, 'subsample': 0.7539605610164881, 'colsample_bytree': 0.9127666333513027, 'gamma': 0.7246502134650312, 'min_child_weight': 9.561299240038938, 'lambda': 6.481100222139939, 'alpha': 8.962487815500863}. Best is trial 0 with value: 0.18295848390224673.


...

[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.15336
[3]	validation-error:0.07750
[4]	validation-error:0.06469
[5]	validation-error:0.05920


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:13:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[6]	validation-error:0.05684
[7]	validation-error:0.05348


[I 2024-04-02 15:13:36,744] Trial 99 pruned. Trial was pruned at iteration 8.


Best Hyperparameters (33 Features):  {'eta': 0.020021417001006916, 'max_depth': 10, 'subsample': 0.9317164208904121, 'colsample_bytree': 0.8743529324059651, 'gamma': 1.1763260217082239, 'min_child_weight': 6.562906540788798, 'lambda': 5.12238113525993, 'alpha': 0.5596591615791868}
Best Error (33 Features):  0.18903726494787432


In [8]:
print(best_params_12)
print(best_params_33)

{'eta': 0.21019153698092585, 'max_depth': 10, 'subsample': 0.9321646856091291, 'colsample_bytree': 0.8325501350820024, 'gamma': 0.7374044077961104, 'min_child_weight': 5.5166658745205455, 'lambda': 3.895007719834459, 'alpha': 4.359152785531455}
{'eta': 0.020021417001006916, 'max_depth': 10, 'subsample': 0.9317164208904121, 'colsample_bytree': 0.8743529324059651, 'gamma': 1.1763260217082239, 'min_child_weight': 6.562906540788798, 'lambda': 5.12238113525993, 'alpha': 0.5596591615791868}


#### Model Training

In [9]:
from sklearn.model_selection import KFold
import optuna

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

# Set Hyperparameters
params_12 = { 'objective': 'binary:hinge',
              'eta': 0.2769198716475172, 
              'max_depth': 10, 
              'subsample': 0.82358642201105, 
              'colsample_bytree': 0.8286208231929323, 
              'gamma': 0.7766612702667438, 
              'min_child_weight': 8.319434489010376, 
              'lambda': 4.43143014244566, 
              'alpha': 4.399273966701367,
              'eval_metric': 'error'
              }

best_params_12['objective'] = 'binary:hinge'
best_params_12['eval_metric'] = 'error'

best_params_33['objective'] = 'binary:hinge'
best_params_33['eval_metric'] = 'error'

# Convert the data into DMatrix format
dtrain_12 = DMatrix(X_train_12, label=y_train)
dvalid_12 = DMatrix(X_test_12, label=y_test)

dtrain_33 = DMatrix(X_train_33, label=y_train)
dvalid_33 = DMatrix(X_test_33, label=y_test)

# Train the Model
xgb_classifier_12 = train(best_params_12, dtrain_12, num_boost_round=3000)
y_pred_12 = xgb_classifier_12.predict(dvalid_12)

print("Model with 12 Features Done.")

xgb_classifier_33 = train(best_params_33, dtrain_33, num_boost_round=3000)
y_pred_33 = xgb_classifier_33.predict(dvalid_33)

print("Model with 33 Features Done.")

Model with 12 Features Done.
Model with 33 Features Done.


#### Evaluation

In [10]:
# Classification Report
print(classification_report(y_test, y_pred_12))
print(classification_report(y_test, y_pred_33))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97     23942
           1       0.95      0.93      0.94     12541

    accuracy                           0.96     36483
   macro avg       0.96      0.95      0.95     36483
weighted avg       0.96      0.96      0.96     36483

              precision    recall  f1-score   support

           0       0.97      0.98      0.98     23942
           1       0.96      0.95      0.95     12541

    accuracy                           0.97     36483
   macro avg       0.97      0.96      0.96     36483
weighted avg       0.97      0.97      0.97     36483



In [11]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)
disp.plot()
plt.show()'''

'# Confusion Matrix for 12 Features\ncm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)\ndisp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)\ndisp.plot()\nplt.show()'

In [12]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''

"# Cross Validation Score\nscores = cross_val_score(XGBClassifier(**params_gbm),\n                        X_train, y_train, scoring='accuracy', cv=cv).mean()\n\nprint(scores)"

In [13]:
# Dumping the model
joblib.dump(xgb_classifier_12, 'xgb_ffs_12.sav')
joblib.dump(xgb_classifier_33, 'xgb_ffs_33.sav')

['xgb_ffs_33.sav']

In [19]:
import content_generator_25
import content_generator_65
import time

def xgb_predict_maliciousness_12(url):

    numerical_values = content_generator_25.feature_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_12.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"
        
def xgb_predict_maliciousness_33(url):

    numerical_values = content_generator_65.feature_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_33.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "www.facebook.com/"
print("Current URL: "+url)

start = time.perf_counter()
prediction = xgb_predict_maliciousness_12(url)
end = time.perf_counter()
print("------- 12 Features -------------")
print(prediction)
print(end-start)

start = time.perf_counter()
prediction = xgb_predict_maliciousness_33(url)
end = time.perf_counter()
print("------- 33 Features -------------")
print(prediction)
print(end-start)

Current URL: www.facebook.com/
------- 12 Features -------------
Benign
0.6155820999993011
------- 33 Features -------------
Benign
0.38702309993095696
